In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Mapa Interativo: Uso do Solo + Valores do Solo - Londrina
Camada 1: Classes de uso do solo (patches 224x224 pixels)
Camada 2: Valores do solo preditos (quadrados 109.45m × 109.45m)
"""

import pandas as pd
import numpy as np
import folium
from folium import plugins
from shapely.geometry import Polygon
from pyproj import Transformer
import matplotlib.colors as mcolors

# ============================================================================
# 1. DEFINIÇÃO DE CORES E PALETAS
# ============================================================================

# Cores para classes de uso do solo
class_colors = {
    'bare': '#bf8040',        # Marrom (solo exposto)
    'bush': '#009900',        # Verde médio
    'crop': '#ffdf99',        # Amarelo claro (agricultura)
    'grass': '#33ff33',       # Verde claro/lima
    'hduf': '#69000d',        # Vermelho escuro (alta densidade urbana)
    'industrial': '#e93529',  # Vermelho alaranjado (industrial)
    'lduf': '#c7171c',        # Vermelho médio (baixa densidade urbana)
    'mduf': '#a10e15',        # Vermelho médio-escuro (média densidade urbana)
    'tree': '#003300',        # Verde escuro
    'water': '#0b9fd5'        # Azul (água)
}

# Paleta tim.colors() do R para valores do solo
def tim_colors(n=1000):
    """Recria a paleta tim.colors() do R"""
    colors = [
        '#000080',  # azul marinho
        '#0000CC',  # azul escuro
        '#0040FF',  # azul
        '#0080FF',  # azul claro
        '#00BFFF',  # azul céu
        '#00FFFF',  # ciano
        '#00FFBF',  # ciano-verde
        '#00FF80',  # verde-ciano
        '#00FF40',  # verde claro
        '#00FF00',  # verde
        '#40FF00',  # verde-amarelo
        '#80FF00',  # amarelo-verde
        '#BFFF00',  # amarelo claro
        '#FFFF00',  # amarelo
        '#FFBF00',  # amarelo-laranja
        '#FF8000',  # laranja
        '#FF4000',  # laranja-vermelho
        '#FF0000'   # vermelho
    ]
    
    cmap = mcolors.LinearSegmentedColormap.from_list("tim", colors, N=n)
    return [mcolors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, n)]

pal_valores = tim_colors(1000)

# ============================================================================
# 2. FUNÇÕES AUXILIARES
# ============================================================================

def parse_coordinates(coord_str):
    """Extrai lat, lon de string 'lat,lon'"""
    lat, lon = map(float, coord_str.split(','))
    return lat, lon

def create_patch_polygon(row):
    """Cria polígono do patch a partir dos 4 vértices"""
    tl_lat, tl_lon = parse_coordinates(row['top_left'])
    tr_lat, tr_lon = parse_coordinates(row['top_right'])
    bl_lat, bl_lon = parse_coordinates(row['bottom_left'])
    br_lat, br_lon = parse_coordinates(row['bottom_right'])
    
    # Folium usa [lat, lon]
    return [
        [tl_lat, tl_lon],  # top_left
        [tr_lat, tr_lon],  # top_right
        [br_lat, br_lon],  # bottom_right
        [bl_lat, bl_lon],  # bottom_left
        [tl_lat, tl_lon]   # fecha o polígono
    ]

def create_square_utm(utm_x, utm_y, half_size):
    """Cria um quadrado em coordenadas UTM"""
    return Polygon([
        (utm_x - half_size, utm_y - half_size),  # inferior esquerdo
        (utm_x + half_size, utm_y - half_size),  # inferior direito
        (utm_x + half_size, utm_y + half_size),  # superior direito
        (utm_x - half_size, utm_y + half_size),  # superior esquerdo
        (utm_x - half_size, utm_y - half_size)   # fecha o polígono
    ])

def transform_polygon_to_latlon(polygon_utm, transformer):
    """Converte polígono UTM para lat/lon"""
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])  # Folium usa [lat, lon]
    
    return coords_latlon

# ============================================================================
# 3. CARREGAR DADOS
# ============================================================================

print("Carregando dados...")

# Dados de uso do solo de Londrina, Ibiporã e Cambé
df_uso_lda = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_londrina/inference_results_with_positions_20250609_083252.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_lda):,} patches")

df_uso_ibip = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_ibipora/inference_results_with_positions_20250609_164529.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_ibip):,} patches")

df_uso_camb = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambe/inference_results_with_positions_20250609_161440.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_camb):,} patches")

# Concatenar dataframes de uso do solo
df_uso = pd.concat([df_uso_lda, df_uso_ibip, df_uso_camb], ignore_index=True)
print(f"✓ Total de uso do solo combinado: {len(df_uso):,} patches")

# Dados de valores do solo de Londrina, Ibiporã e Cambé
df_val_lda = pd.read_csv('/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/mediana/predicoes_unit_Londrina.csv')
print(f"✓ Valores do solo: {len(df_val_lda):,} pontos")
df_val_ibip = pd.read_csv('/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/mediana/predicoes_unit_Ibipora.csv')
print(f"✓ Valores do solo: {len(df_val_ibip):,} pontos")
df_val_camb = pd.read_csv('/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/mediana/predicoes_unit_Cambe.csv')
print(f"✓ Valores do solo: {len(df_val_camb):,} pontos")

# Concatenar dataframes de valores do solo
df_valor = pd.concat([df_val_lda, df_val_ibip, df_val_camb], ignore_index=True)
print(f"✓ Total de valores do solo combinado: {len(df_valor):,} pontos")

# ============================================================================
# 4. PREPARAR CAMADA 1: USO DO SOLO
# ============================================================================

print("\nPreparando camada de uso do solo...")

features_uso = []
for idx, row in df_uso.iterrows():
    if idx % 1000 == 0:
        print(f"  Processando patch {idx+1}/{len(df_uso)}")
    
    polygon_coords = create_patch_polygon(row)
    predicted_class = row['predicted_class']
    color = class_colors.get(predicted_class, '#808080')  # cinza como fallback
    confidence = row['prediction_confidence']
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.6,
            "class": predicted_class,
            "confidence": f"{confidence:.2%}",
            "popup": f"Land Use: {predicted_class}<br>Confidence: {confidence:.2%}"
        }
    }
    features_uso.append(feature)

geojson_uso = {
    "type": "FeatureCollection",
    "features": features_uso
}

print(f"✓ Criados {len(features_uso):,} polígonos de uso do solo")

# ============================================================================
# 5. PREPARAR CAMADA 2: VALORES DO SOLO
# ============================================================================

print("\nPreparando camada de valores do solo...")

# Parâmetros do grid
grid_spacing = 109.45  # metros
half_spacing = grid_spacing / 2

# Configurar transformador de coordenadas
transformer = Transformer.from_crs("EPSG:29192", "EPSG:4326", always_xy=True)

# Criar quadrados em UTM
utm_polygons = []
for idx, row in df_valor.iterrows():
    if idx % 1000 == 0:
        print(f"  Criando quadrado {idx+1}/{len(df_valor)}")
    square_utm = create_square_utm(row['utm_x'], row['utm_y'], half_spacing)
    utm_polygons.append(square_utm)

print(f"✓ Criados {len(utm_polygons):,} quadrados em UTM")

# Converter para lat/lon
latlon_polygons = []
for idx, polygon_utm in enumerate(utm_polygons):
    if idx % 1000 == 0:
        print(f"  Convertendo para lat/lon {idx+1}/{len(utm_polygons)}")
    latlon_polygon = transform_polygon_to_latlon(polygon_utm, transformer)
    latlon_polygons.append(latlon_polygon)

print(f"✓ Convertidos {len(latlon_polygons):,} quadrados para lat/lon")

# Obter min e max dos valores preditos (em log)
min_log_price = df_valor['price_predicted'].min()
max_log_price = df_valor['price_predicted'].max()
price_range = max_log_price - min_log_price

# Criar breaks baseados no range dos dados
breaks = np.linspace(min_log_price, max_log_price, 18)

def get_color_from_price(price):
    """Obtém cor da paleta baseada no preço"""
    normalized = (price - min_log_price) / price_range
    color_index = int(normalized * (len(pal_valores) - 1))
    color_index = max(0, min(color_index, len(pal_valores) - 1))
    return pal_valores[color_index]

# Criar features GeoJSON para valores
features_valor = []
for idx in range(len(latlon_polygons)):
    if idx % 1000 == 0:
        print(f"  Criando feature GeoJSON {idx+1}/{len(latlon_polygons)}")
    
    polygon_coords = latlon_polygons[idx]
    price_log = df_valor.iloc[idx]['price_predicted']
    price_real = np.exp(price_log)
    color = get_color_from_price(price_log)
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.7,
            "price": f"R$ {price_real:,.2f}/m²",
            "popup": f"Predicted Unit Land Value<br>R$ {price_real:,.2f}/m²"
        }
    }
    features_valor.append(feature)

geojson_valor = {
    "type": "FeatureCollection",
    "features": features_valor
}

print(f"✓ Criados {len(features_valor):,} polígonos de valores")

# ============================================================================
# 6. CRIAR MAPA INTERATIVO
# ============================================================================

print("\nCriando mapa interativo...")

# Calcular centro do mapa
center_lat = df_uso['center'].apply(lambda x: parse_coordinates(x)[0]).mean()
center_lon = df_uso['center'].apply(lambda x: parse_coordinates(x)[1]).mean()

# Criar mapa base com satélite
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)

# Adicionar camada de uso do solo
layer_uso = folium.FeatureGroup(name='Land Use Classes', show=True)
folium.GeoJson(
    geojson_uso,
    style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color': feature['properties']['color'],
        'weight': feature['properties']['weight'],
        'fillOpacity': feature['properties']['fillOpacity'],
        'opacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['popup'],
        aliases=[''],
        labels=False
    )
).add_to(layer_uso)
layer_uso.add_to(m)

# Adicionar camada de valores do solo
layer_valor = folium.FeatureGroup(name='Land Values', show=True)
folium.GeoJson(
    geojson_valor,
    style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color': feature['properties']['color'],
        'weight': feature['properties']['weight'],
        'fillOpacity': feature['properties']['fillOpacity'],
        'opacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['popup'],
        aliases=[''],
        labels=False
    )
).add_to(layer_valor)
layer_valor.add_to(m)

# Adicionar controle de camadas
folium.LayerControl(position='topleft', collapsed=False).add_to(m)

print("✓ Camadas adicionadas ao mapa")

# ============================================================================
# 7. CRIAR LEGENDAS
# ============================================================================

print("Criando legendas...")

# Legenda de uso do solo
legend_uso_html = """
<div style="position: fixed; 
           bottom: 50px; left: 10px; width: 200px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:12px; padding: 10px;">
<p style="margin:0 0 8px 0;"><b>Land Use Classes</b></p>
<table style="font-size:11px; width:100%;">
"""

# Adicionar todas as 10 classes com labels adequados
all_classes = {
    'bare': 'Bare Soil',
    'bush': 'Bush',
    'crop': 'Cropland',
    'grass': 'Grassland',
    'hduf': 'High Density Urban',
    'industrial': 'Industrial',
    'lduf': 'Low Density Urban',
    'mduf': 'Medium Density Urban',
    'tree': 'Tree',
    'water': 'Water'
}

for class_key, class_label in all_classes.items():
    color = class_colors[class_key]
    legend_uso_html += f"""
    <tr>
        <td style="background-color:{color}; width:20px; height:15px; border:1px solid #666;"></td>
        <td style="padding-left:8px; vertical-align:middle;">{class_label}</td>
    </tr>
    """

legend_uso_html += """
</table>
</div>
"""

# Legenda de valores do solo
legend_valor_html = f"""
<div style="position: fixed; 
           top: 10px; right: 10px; width: 260px; height: 550px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:12px; padding: 10px; overflow-y: scroll;">
<p style="margin:0 0 8px 0;"><b>Predicted Unit Land Value<br>(R$/m²)</b></p>
<table style="font-size:10px; border-collapse: collapse; width:100%;">
"""

# Adicionar faixas de cores na legenda
for i in range(len(breaks)-1):
    color_index = int((i / (len(breaks)-2)) * (len(pal_valores)-1))
    color = pal_valores[color_index]
    
    # Calcular preços reais para as faixas
    price_min = np.exp(breaks[i])
    price_max = np.exp(breaks[i+1])
    
    legend_valor_html += f"""
    <tr>
        <td style="background-color:{color}; width:20px; height:10px; border:1px solid #ccc;"></td>
        <td style="padding-left:8px; vertical-align:middle;">R$ {price_min:,.0f} - R$ {price_max:,.0f}</td>
    </tr>
    """

legend_valor_html += f"""
</table>
<div style="font-size:10px; margin-top:15px; color:#666; line-height:1.4;">
<p style="margin:0; font-weight:bold;">Grid Parameters:</p>
<ul style="margin:5px 0; padding-left:15px;">
    <li>Land Use: 224×224 pixels patches</li>
    <li>Land Value: {grid_spacing:.1f}m × {grid_spacing:.1f}m grid</li>
    <li>Paradigm area: 363m²</li>
</ul>
<p style="margin:10px 0 0 0; font-style:italic; font-size:9px;">
Londrina-PR, Brazil<br>
Data: 2025
</p>
</div>
</div>
"""

# Adicionar legendas ao mapa
m.get_root().html.add_child(folium.Element(legend_uso_html))
m.get_root().html.add_child(folium.Element(legend_valor_html))

print("✓ Legendas criadas")

# ============================================================================
# 8. SALVAR MAPA
# ============================================================================

output_file = '/Users/fjcosta/Documents/landCoverlandValue/landcover_landvalue/mapa_lda_camb_ibip_uso_valor_solo.html'
m.save(output_file)

print(f"\n{'='*70}")
print(f"✅ MAPA CRIADO COM SUCESSO!")
print(f"{'='*70}")
# print(f"Arquivo: {output_file}")
print(f"\nCamadas:")
print(f"  • Land Use Classes: {len(features_uso):,} patches")
print(f"  • Land Values: {len(features_valor):,} grid cells")
print(f"\nUse o controle de camadas no mapa para ligar/desligar cada camada.")
print(f"{'='*70}")

Carregando dados...
✓ Uso do solo: 18,302 patches
✓ Uso do solo: 3,903 patches
✓ Uso do solo: 5,905 patches
✓ Total de uso do solo combinado: 28,110 patches
✓ Valores do solo: 18,384 pontos
✓ Valores do solo: 3,916 pontos
✓ Valores do solo: 5,922 pontos
✓ Total de valores do solo combinado: 28,222 pontos

Preparando camada de uso do solo...
  Processando patch 1/28110
  Processando patch 1001/28110
  Processando patch 2001/28110
  Processando patch 3001/28110
  Processando patch 4001/28110
  Processando patch 5001/28110
  Processando patch 6001/28110
  Processando patch 7001/28110
  Processando patch 8001/28110
  Processando patch 9001/28110
  Processando patch 10001/28110
  Processando patch 11001/28110
  Processando patch 12001/28110
  Processando patch 13001/28110
  Processando patch 14001/28110
  Processando patch 15001/28110
  Processando patch 16001/28110
  Processando patch 17001/28110
  Processando patch 18001/28110
  Processando patch 19001/28110
  Processando patch 20001/2811